In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import json
import re 
import time
from pandas.io.json import json_normalize

You need to add another data set to the input data.

1. kernel Top right "+ Add data"
2. search dataset "lol"
3. Add "League of Legends(LOL) - Ranked Games 2020"

# data preprocessing

# data load
    challengergame_df = pd.read_csv('../input/league-of-legendslol-ranked-games-2020-ver1/challenger_match.csv')
    challengergame_df2 = pd.read_csv('../input/league-of-legendslol-ranked-games-2020-ver1/challenger_match_V2.csv')

    ingame_df = pd.read_pickle('../input/league-of-legendslol-ranked-games-2020-ver1/match_data_version2.pickle')

# data concat
    chall_game = pd.concat([challengergame_df,challengergame_df2],axis = 0)
# extract unique gameid
    chall_game_unique = chall_game.drop_duplicates('gameId')

# gameid astype

    ingame_df['gameId'] = ingame_df['gameId'].astype('int')

# extract only challenger game
    chall_match = pd.merge(ingame_df,chall_game_unique[['gameId','accountId']], on = 'gameId', how = 'left')\
                                        .drop(columns=['status.message','status.status_code'])\
                                        .dropna(axis=0)
    len(chall_match)

    chall_match.index = range(len(chall_match))

# challenger game summary

    columns_ls = ['gameId','gameDuraton','blueWins','blueFirstBlood','blueFirstTower',\
                 'blueFirstBaron','blueFirstDragon','blueFirstInhibitor','blueDragonKills',\
                 'blueBaronKills','blueTowerKills','blueInhibitorKills','blueWardPlaced',\
                 'blueWardkills','blueKills','blueDeath','blueAssist','blueChampionDamageDealt',\
                 'blueTotalGold','blueTotalMinionKills','blueTotalLevel','blueAvgLevel',\
                 'blueJungleMinionKills','blueKillingSpree','blueTotalHeal','blueObjectDamageDealt',\
                 'redWins','redFirstBlood','redFirstTower',\
                 'redFirstBaron','redFirstDragon','redFirstInhibitor','redDragonKills',\
                 'redBaronKills','redTowerKills','redInhibitorKills','redWardPlaced',\
                 'redWardkills','redKills','redDeath','redAssist','redChampionDamageDealt',\
                 'redTotalGold','redTotalMinionKills','redTotalLevel','redAvgLevel',\
                 'redJungleMinionKills','redKillingSpree','redTotalHeal','redObjectDamageDealt']
    challenger_game_info = pd.DataFrame()

    for x in range(len(chall_match)):

        if x % 100 == 0:
            print(str(x) + ' ~ ' + str(x+100) + ' processing')
        gameId = chall_match['gameId'].iloc[x]
        gameDuration = chall_match['gameDuration'].iloc[x]

        for k in range(len(chall_match['teams'].iloc[x])):

            if chall_match['teams'].iloc[x][k]['teamId'] == 100:
                win_tf = chall_match['teams'].iloc[x][k]['win']
                firstblood = chall_match['teams'].iloc[x][k]['firstBlood']
                firstTower = chall_match['teams'].iloc[x][k]['firstTower']
                firstBaron = chall_match['teams'].iloc[x][k]['firstBaron']
                firstDragon = chall_match['teams'].iloc[x][k]['firstDragon']
                firstinhibitor = chall_match['teams'].iloc[x][k]['firstInhibitor']

                dragonkill = chall_match['teams'].iloc[x][k]['dragonKills']
                baronkill = chall_match['teams'].iloc[x][k]['baronKills']
                towerkill = chall_match['teams'].iloc[x][k]['towerKills']
                inhibitorkill = chall_match['teams'].iloc[x][k]['inhibitorKills']
            else:
                win_tf2 = chall_match['teams'].iloc[x][k]['win']
                firstblood2 = chall_match['teams'].iloc[x][k]['firstBlood']
                firstTower2 = chall_match['teams'].iloc[x][k]['firstTower']
                firstBaron2 = chall_match['teams'].iloc[x][k]['firstBaron']
                firstDragon2 = chall_match['teams'].iloc[x][k]['firstDragon']
                firstinhibitor2 = chall_match['teams'].iloc[x][k]['firstInhibitor']

                dragonkill2 = chall_match['teams'].iloc[x][k]['dragonKills']
                baronkill2 = chall_match['teams'].iloc[x][k]['baronKills']
                towerkill2 = chall_match['teams'].iloc[x][k]['towerKills']
                inhibitorkill2 = chall_match['teams'].iloc[x][k]['inhibitorKills']

        '''blue team : 100'''
        wardplace = 0
        wardkill = 0
        champkill = 0
        champdeath = 0
        champassist = 0
        damage = 0
        gold = 0
        minion = 0
        level = 0
        jungleminion = 0
        killingspree = 0
        heal = 0
        objectdamage = 0

        '''red team : 200'''
        wardplace2 = 0
        wardkill2 = 0
        champkill2 = 0
        champdeath2 = 0
        champassist2 = 0
        damage2 = 0
        gold2 = 0
        minion2 = 0
        level2 = 0
        jungleminion2 = 0
        killingspree2 = 0
        heal2 = 0
        objectdamage2 = 0
        for i in range(len(chall_match['participants'].iloc[x])):

            if chall_match['participants'].iloc[x][i]['teamId'] == 100:
                try:
                    wardplace0 = chall_match['participants'].iloc[x][i]['stats']['wardsPlaced']
                except:
                    wardplace0 = 0

                try:
                    wardkill0 = chall_match['participants'].iloc[x][i]['stats']['wardsKilled']
                except:
                    wardkill0 = 0
                try:
                    champkill0 = chall_match['participants'].iloc[x][i]['stats']['kills']
                except:
                    champkill0 = 0
                try:
                    champdeath0 = chall_match['participants'].iloc[x][i]['stats']['deaths']
                except:
                    champdeath0 = 0
                try:
                    champassist0 = chall_match['participants'].iloc[x][i]['stats']['assists']
                except:
                    champassist0 = 0
                try:
                    damage0 = chall_match['participants'].iloc[x][i]['stats']['totalDamageDealtToChampions']
                except:
                    damage0 = 0
                try:
                    gold0 = chall_match['participants'].iloc[x][i]['stats']['goldEarned']
                except:
                    gold0 = 0
                try:
                    minion0 = chall_match['participants'].iloc[x][i]['stats']['totalMinionsKilled']
                except:
                    minion0 = 0
                try:
                    level0 = chall_match['participants'].iloc[x][i]['stats']['champLevel']
                except:
                    level0 = 0
                try:
                    jungleminion0 = chall_match['participants'].iloc[x][i]['stats']['neutralMinionsKilled']
                except:
                    jungleminion0 = 0
                try:
                    killingspree0 = chall_match['participants'].iloc[x][i]['stats']['killingSprees']
                except:
                    killingspree0 = 0
                try:
                    heal0 = chall_match['participants'].iloc[x][i]['stats']['totalHeal']
                except:
                    heal0 = 0
                try:
                    objectdamage0 = chall_match['participants'].iloc[x][i]['stats']['damageDealtToObjectives']
                except:
                    objectdamage0 = 0

                wardplace += wardplace0
                wardkill += wardkill0
                champkill += champkill0
                champdeath += champdeath0
                champassist += champassist0
                damage += damage0
                gold += gold0
                minion += minion0
                level += level0
                jungleminion += jungleminion0
                killingspree += killingspree0
                heal += heal0
                objectdamage += objectdamage0
            else:

                try:
                    wardplace0 = chall_match['participants'].iloc[x][i]['stats']['wardsPlaced']
                except:
                    wardplace0 = 0

                try:
                    wardkill0 = chall_match['participants'].iloc[x][i]['stats']['wardsKilled']
                except:
                    wardkill0 = 0
                try:
                    champkill0 = chall_match['participants'].iloc[x][i]['stats']['kills']
                except:
                    champkill0 = 0
                try:
                    champdeath0 = chall_match['participants'].iloc[x][i]['stats']['deaths']
                except:
                    champdeath0 = 0
                try:
                    champassist0 = chall_match['participants'].iloc[x][i]['stats']['assists']
                except:
                    champassist0 = 0
                try:
                    damage0 = chall_match['participants'].iloc[x][i]['stats']['totalDamageDealtToChampions']
                except:
                    damage0 = 0
                try:
                    gold0 = chall_match['participants'].iloc[x][i]['stats']['goldEarned']
                except:
                    gold0 = 0
                try:
                    minion0 = chall_match['participants'].iloc[x][i]['stats']['totalMinionsKilled']
                except:
                    minion0 = 0
                try:
                    level0 = chall_match['participants'].iloc[x][i]['stats']['champLevel']
                except:
                    level0 = 0
                try:
                    jungleminion0 = chall_match['participants'].iloc[x][i]['stats']['neutralMinionsKilled']
                except:
                    jungleminion0 = 0
                try:
                    killingspree0 = chall_match['participants'].iloc[x][i]['stats']['killingSprees']
                except:
                    killingspree0 = 0
                try:
                    heal0 = chall_match['participants'].iloc[x][i]['stats']['totalHeal']
                except:
                    heal0 = 0
                try:
                    objectdamage0 = chall_match['participants'].iloc[x][i]['stats']['damageDealtToObjectives']
                except:
                    objectdamage0 = 0


                wardplace2 += wardplace0
                wardkill2 += wardkill0
                champkill2 += champkill0
                champdeath2 += champdeath0
                champassist2 += champassist0
                damage2 += damage0
                gold2 += gold0
                minion2 += minion0
                level2 += level0
                jungleminion2 += jungleminion0
                killingspree2 += killingspree0
                heal2 += heal0
                objectdamage2 += objectdamage0


        total_result = [int(gameId),int(gameDuration)]

        blue_object = [win_tf,firstblood,firstTower,firstBaron,firstDragon,firstinhibitor,int(dragonkill)\
                      ,int(baronkill),int(towerkill),int(inhibitorkill)]

        red_object = [win_tf2,firstblood2,firstTower2,firstBaron2,firstDragon2,firstinhibitor2,int(dragonkill2)\
                      ,int(baronkill2),int(towerkill2),int(inhibitorkill2)]

        blue_team_data = [int(wardplace),int(wardkill),int(champkill),int(champdeath),int(champassist),int(damage),int(gold),int(minion)\
                         ,int(level),float(level/5),int(jungleminion),int(killingspree),int(heal),int(objectdamage)]

        red_team_data = [int(wardplace2),int(wardkill2),int(champkill2),int(champdeath2),int(champassist2),int(damage2),int(gold2),int(minion2)\
                         ,int(level2),float(level2/5),int(jungleminion2),int(killingspree2),int(heal2),int(objectdamage2)]


        data_value = np.array([total_result + blue_object + blue_team_data + red_object + red_team_data])

        challenger_game_info0 = pd.DataFrame(data_value,columns = columns_ls)

        challenger_game_info = challenger_game_info.append(challenger_game_info0)

# datatype convert
# Because array type

    numeric_col = ['gameId','gameDuraton','blueDragonKills',\
                 'blueBaronKills','blueTowerKills','blueInhibitorKills','blueWardPlaced',\
                 'blueWardkills','blueKills','blueDeath','blueAssist','blueChampionDamageDealt',\
                 'blueTotalGold','blueTotalMinionKills','blueTotalLevel',\
                 'blueJungleMinionKills','blueKillingSpree','blueTotalHeal','blueObjectDamageDealt',\
                 'redDragonKills',\
                 'redBaronKills','redTowerKills','redInhibitorKills','redWardPlaced',\
                 'redWardkills','redKills','redDeath','redAssist','redChampionDamageDealt',\
                 'redTotalGold','redTotalMinionKills','redTotalLevel',\
                 'redJungleMinionKills','redKillingSpree','redTotalHeal','redObjectDamageDealt']

    object_col = ['blueWins','blueFirstBlood','blueFirstTower',\
                 'blueFirstBaron','blueFirstDragon','blueFirstInhibitor',\
                 'redWins','redFirstBlood','redFirstTower',\
                 'redFirstBaron','redFirstDragon','redFirstInhibitor']

    for i in numeric_col:
        challenger_game_info[i] = challenger_game_info[i].astype('int')

    for i in ['blueAvgLevel','redAvgLevel']:
        challenger_game_info[i] = challenger_game_info[i].astype('float')

    mapping1 = {'Win':1,'Fail':0}
    mapping2 = {'True':1,'False':0}

    for i in object_col:
        if i in ['blueWins','redWins']:
            challenger_game_info[i] = challenger_game_info[i].map(mapping1).tolist()
        else:
            challenger_game_info[i] = challenger_game_info[i].map(mapping2).tolist()

    challenger_game_info.info()

    challenger_game_info_test.head()

# Next EDA